In [1]:
!python generate_ds.py --total_samples 1000 --seed 26 --offset 0 --model_dir robot_reboot_model/model_0/

INFO:root:Generating 1000 samples with seed 26
INFO:root:Working with 12 cores
INFO:root:Processing jobs
2021-04-03 22:38:07.371107: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-03 22:38:07.374485: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-03 22:38:07.375292: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-03 22:38:07.375967: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-03 22:38:07.376414: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Netw

2021-04-03 22:38:07.655296: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_0.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_3.tfrecords
INFO:root:Saving file data/robot_reboot_4.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_11.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_6.tfrecords
INFO:root:Search f

2021-04-03 22:52:15.479101: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-03 22:52:15.479968: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-03 22:52:15.480423: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-03 22:52:15.480880: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-03 22:52:15.481640: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-03 22:52:15.483179: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-03 22:52:15.483985: I tensorflow/compiler/mlir/mlir_grap

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created

...

2021-04-04 14:11:34.458467: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-04 14:11:34.459702: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a da

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Game created

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created

INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_996.tfrecords
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_997.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_999.tfrecords
INFO:root:Finished processed
INFO:root:Finished after 58964.77634310722 seconds ---


In [2]:
!python generate_ds.py --total_samples 1000 --seed 18 --offset 1000 --model_dir robot_reboot_model/model_0/

INFO:root:Generating 1000 samples with seed 18
INFO:root:Working with 12 cores
INFO:root:Processing jobs
2021-04-04 15:00:56.329862: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-04 15:00:56.330295: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-04 15:00:56.331022: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-04 15:00:56.331302: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operati

2021-04-04 15:00:56.770548: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_1009.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_1003.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_1007.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_1008.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_1002.tfrecords
INFO

2021-04-04 15:13:55.709798: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-04 15:13:55.710651: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-04 15:13:55.710664: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-04 15:13:55.710852: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-04 15:13:55.711905: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-04 15:13:55.712895: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-04 15:13:55.713023: I tensorflow/compiler/mlir/mlir_grap

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

...

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_1999.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_1996.tfrecords
INFO:root:Finished processed
INFO:root:Finished after 59525.61656808853 seconds ---


In [ ]:
!python generate_ds.py --total_samples 1000 --seed 28 --offset 2000 --model_dir robot_reboot_model/model_0/

INFO:root:Generating 1000 samples with seed 28
INFO:root:Working with 12 cores
INFO:root:Processing jobs
2021-04-05 19:01:56.537323: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-05 19:01:56.537613: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-05 19:01:56.537332: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-05 19:01:56.538900: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-05 19:01:56.539536: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-0

2021-04-05 19:01:56.903115: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_2000.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_2001.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_2005.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_2003.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_2010.tfrecords
INFO

2021-04-05 19:12:34.299338: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-05 19:12:34.302457: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-05 19:12:34.304263: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-05 19:12:34.304379: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-05 19:12:34.307455: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-05 19:12:34.308725: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-05 19:12:34.309934: I tensorflow/compiler/mlir/mlir_grap

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:

...

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:

2021-04-06 06:36:33.566008: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:r

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created

INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

In [4]:
#!python generate_ds.py --total_samples 1000 --seed 4 --offset 3000 --model_dir robot_reboot_model/model_0/

INFO:root:Generating 1000 samples with seed 4
INFO:root:Working with 12 cores
INFO:root:Processing jobs
2021-04-05 07:33:15.185553: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-05 07:33:15.185893: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-05 07:33:15.186293: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-05 07:33:15.186406: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-05 07:33:15.186510: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-05

^C
Process Process-5:
Traceback (most recent call last):
  File "generate_ds.py", line 86, in <module>
Process Process-11:
Process Process-4:
Process Process-3:
Process Process-6:
Process Process-2:
    create_tfrecord(args.offset, args.total_samples, args.model_dir)
  File "generate_ds.py", line 44, in create_tfrecord
    [t.join() for t in processes]
  File "generate_ds.py", line 44, in <listcomp>
    [t.join() for t in processes]
  File "/usr/local/opt/python/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/usr/local/opt/python/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/local/opt/python/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt
Process P